In [1]:
import numpy as np
from copy import deepcopy
from tqdm import tqdm
import random
from typing import Any, Tuple, Optional, Sequence

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.nn import SAGEConv
from torch_geometric.loader import DataLoader
from torch.autograd import Function

/home/hhchung/dyngraph-uda/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from sklearn.metrics import f1_score

In [4]:
from dataset import load_nc_dataset

In [5]:
import matplotlib.pyplot as plt

In [6]:
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

## Model Structure ##

In [7]:
class TwoLayerGraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, out_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        return x
    
class MLPHead(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.linear1 = nn.Linear(in_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, out_dim)
    
    def forward(self, x):
        x1 = self.linear1(x)
        x1 = F.elu(x1)
        x1 = F.dropout(x1, p=self.dropout)
        output = self.linear2(x1)
        # x = F.softmax(x, dim=1)
        features = x1
        return output, features


In [8]:
class GaussianKernel(nn.Module):
    r"""Gaussian Kernel Matrix
    Gaussian Kernel k is defined by
    .. math::
        k(x_1, x_2) = \exp \left( - \dfrac{\| x_1 - x_2 \|^2}{2\sigma^2} \right)
    where :math:`x_1, x_2 \in R^d` are 1-d tensors.
    Gaussian Kernel Matrix K is defined on input group :math:`X=(x_1, x_2, ..., x_m),`
    .. math::
        K(X)_{i,j} = k(x_i, x_j)
    Also by default, during training this layer keeps running estimates of the
    mean of L2 distances, which are then used to set hyperparameter  :math:`\sigma`.
    Mathematically, the estimation is :math:`\sigma^2 = \dfrac{\alpha}{n^2}\sum_{i,j} \| x_i - x_j \|^2`.
    If :attr:`track_running_stats` is set to ``False``, this layer then does not
    keep running estimates, and use a fixed :math:`\sigma` instead.
    Args:
        sigma (float, optional): bandwidth :math:`\sigma`. Default: None
        track_running_stats (bool, optional): If ``True``, this module tracks the running mean of :math:`\sigma^2`.
          Otherwise, it won't track such statistics and always uses fix :math:`\sigma^2`. Default: ``True``
        alpha (float, optional): :math:`\alpha` which decides the magnitude of :math:`\sigma^2` when track_running_stats is set to ``True``
    Inputs:
        - X (tensor): input group :math:`X`
    Shape:
        - Inputs: :math:`(minibatch, F)` where F means the dimension of input features.
        - Outputs: :math:`(minibatch, minibatch)`
    """

    def __init__(self, sigma: Optional[float] = None, track_running_stats: Optional[bool] = True,
                 alpha: Optional[float] = 1.):
        super(GaussianKernel, self).__init__()
        assert track_running_stats or sigma is not None
        self.sigma_square = torch.tensor(sigma * sigma) if sigma is not None else None
        self.track_running_stats = track_running_stats
        self.alpha = alpha

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        l2_distance_square = ((X.unsqueeze(0) - X.unsqueeze(1)) ** 2).sum(2)

        if self.track_running_stats:
            self.sigma_square = self.alpha * torch.mean(l2_distance_square.detach())

        return torch.exp(-l2_distance_square / (2 * self.sigma_square))
    
class JointMultipleKernelMaximumMeanDiscrepancy(nn.Module):
    r"""
    Args:
        kernels (tuple(tuple(torch.nn.Module))): kernel functions, where `kernels[r]` corresponds to kernel :math:`k^{\mathcal{L}[r]}`.
        linear (bool): whether use the linear version of JAN. Default: False
        thetas (list(Theta): use adversarial version JAN if not None. Default: None
    Inputs:
        - z_s (tuple(tensor)): multiple layers' activations from the source domain, :math:`z^s`
        - z_t (tuple(tensor)): multiple layers' activations from the target domain, :math:`z^t`
    Shape:
        - :math:`z^{sl}` and :math:`z^{tl}`: :math:`(minibatch, *)`  where * means any dimension
        - Outputs: scalar
    .. note::
        Activations :math:`z^{sl}` and :math:`z^{tl}` must have the same shape.
    .. note::
        The kernel values will add up when there are multiple kernels for a certain layer.
    Examples::
        >>> feature_dim = 1024
        >>> batch_size = 10
        >>> layer1_kernels = (GaussianKernel(alpha=0.5), GaussianKernel(1.), GaussianKernel(2.))
        >>> layer2_kernels = (GaussianKernel(1.), )
        >>> loss = JointMultipleKernelMaximumMeanDiscrepancy((layer1_kernels, layer2_kernels))
        >>> # layer1 features from source domain and target domain
        >>> z1_s, z1_t = torch.randn(batch_size, feature_dim), torch.randn(batch_size, feature_dim)
        >>> # layer2 features from source domain and target domain
        >>> z2_s, z2_t = torch.randn(batch_size, feature_dim), torch.randn(batch_size, feature_dim)
        >>> output = loss((z1_s, z2_s), (z1_t, z2_t))
    """

    def __init__(self, kernels: Sequence[Sequence[nn.Module]], linear: Optional[bool] = True, thetas: Sequence[nn.Module] = None):
        super(JointMultipleKernelMaximumMeanDiscrepancy, self).__init__()
        self.kernels = kernels
        self.index_matrix = None
        self.linear = linear
        if thetas:
            self.thetas = thetas
        else:
            self.thetas = [nn.Identity() for _ in kernels]

    def forward(self, z_s: torch.Tensor, z_t: torch.Tensor) -> torch.Tensor:
        batch_size = int(z_s[0].size(0))
        self.index_matrix = _update_index_matrix(batch_size, self.index_matrix, self.linear).to(z_s[0].device)

        kernel_matrix = torch.ones_like(self.index_matrix)
        for layer_z_s, layer_z_t, layer_kernels, theta in zip(z_s, z_t, self.kernels, self.thetas):
            layer_features = torch.cat([layer_z_s, layer_z_t], dim=0)
            layer_features = theta(layer_features)
            kernel_matrix *= sum(
                [kernel(layer_features) for kernel in layer_kernels])  # Add up the matrix of each kernel

        # Add 2 / (n-1) to make up for the value on the diagonal
        # to ensure loss is positive in the non-linear version
        loss = (kernel_matrix * self.index_matrix).sum() + 2. / float(batch_size - 1)
        return loss

def _update_index_matrix(batch_size: int, index_matrix: Optional[torch.Tensor] = None,
                         linear: Optional[bool] = True) -> torch.Tensor:
    r"""
    Update the `index_matrix` which convert `kernel_matrix` to loss.
    If `index_matrix` is a tensor with shape (2 x batch_size, 2 x batch_size), then return `index_matrix`.
    Else return a new tensor with shape (2 x batch_size, 2 x batch_size).
    """
    if index_matrix is None or index_matrix.size(0) != batch_size * 2:
        index_matrix = torch.zeros(2 * batch_size, 2 * batch_size)
        if linear:
            for i in range(batch_size):
                s1, s2 = i, (i + 1) % batch_size
                t1, t2 = s1 + batch_size, s2 + batch_size
                index_matrix[s1, s2] = 1. / float(batch_size)
                index_matrix[t1, t2] = 1. / float(batch_size)
                index_matrix[s1, t2] = -1. / float(batch_size)
                index_matrix[s2, t1] = -1. / float(batch_size)
        else:
            for i in range(batch_size):
                for j in range(batch_size):
                    if i != j:
                        index_matrix[i][j] = 1. / float(batch_size * (batch_size - 1))
                        index_matrix[i + batch_size][j + batch_size] = 1. / float(batch_size * (batch_size - 1))
            for i in range(batch_size):
                for j in range(batch_size):
                    index_matrix[i][j + batch_size] = -1. / float(batch_size * batch_size)
                    index_matrix[i + batch_size][j] = -1. / float(batch_size * batch_size)
    return index_matrix
    
class Theta(nn.Module):
    """
    maximize loss respect to :math:`\theta`
    minimize loss respect to features
    """
    def __init__(self, dim: int):
        super(Theta, self).__init__()
        self.grl1 = GradientReverseLayer()
        self.grl2 = GradientReverseLayer()
        self.layer1 = nn.Linear(dim, dim)
        nn.init.eye_(self.layer1.weight)
        nn.init.zeros_(self.layer1.bias)

    def forward(self, features: torch.Tensor) -> torch.Tensor:
        features = self.grl1(features)
        return self.grl2(self.layer1(features))


## Data Preparation ##

* Train: 0-6
* Val: 7, 8
* Test and adapt: 9-13, 14-18, 19-23, 24-28, 29-33, 34-38, 39-43, 44-48

In [9]:
def get_data(data_dir, dataset, sub_dataset=None):
    if dataset == 'elliptic':
        data = load_nc_dataset(data_dir, 'elliptic', sub_dataset)
    else:
        raise ValueError('Invalid dataname')
    # if len(data.y.shape) == 1:
    #     data.y = data.y.unsqueeze(1)
    return data

In [10]:
data_dir = '/home/hhchung/data/graph-data/elliptic_bitcoin_dataset'

## Train / Test / Adapt Loop ##

In [11]:
def train(encoder, mlp, optimizer, loader, loss_fn, device='cpu'):
    encoder.train()
    mlp.train()
    optimizer.zero_grad()
    
    total_train_loss = 0
    for data in loader:
        data = data.to(device)
        out, _ = mlp(encoder(data.x, data.edge_index))
        loss = loss_fn(out[data.mask], data.y[data.mask])
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    
    total_train_loss /= len(loader)
    return total_train_loss

@torch.no_grad()
def test(encoder, mlp, loader, loss_fn, device='cpu'):
    encoder.eval()
    mlp.eval()
    total_val_loss = 0
    total_f1 = 0
    for data in loader:
        data = data.to(device)
        out, _ = mlp(encoder(data.x, data.edge_index))
        loss = loss_fn(out[data.mask], data.y[data.mask])
        y_pred = torch.argmax(out, dim=1)
        f1 = f1_score(y_pred[data.mask].detach().cpu().numpy(), data.y[data.mask].detach().cpu().numpy())
        total_val_loss += loss.item()
        total_f1 += f1
    total_val_loss /= len(loader)
    total_f1 /= len(loader)
    return total_val_loss, total_f1



In [12]:
def minibatch_jmmd(jmmd_loss, src_f, src_y, tgt_f, tgt_y, batch_size=256):
    src_loader = torch.utils.data.DataLoader(tuple(zip(list(src_f), list(src_y))), batch_size=batch_size, shuffle=True)
    tgt_loader = torch.utils.data.DataLoader(tuple(zip(list(tgt_f), list(tgt_y))), batch_size=batch_size, shuffle=True)
    src_iter = iter(src_loader)
    tgt_iter = iter(tgt_loader)
    len_dataloader = min(len(src_loader), len(tgt_loader))
    
    total_transfer_loss = 0
    for i in range(len_dataloader):
        src_f, src_y = src_iter.next()
        tgt_f, tgt_y = tgt_iter.next()
        if src_f.shape[0] != tgt_f.shape[0]:
            break
        
        # if src_f.shape[0] < tgt_f.shape[0]:
        #     src_iter = iter(src_loader)
        #     src_f, src_y = src_iter.next()
        # else:
        #     tgt_iter = iter(tgt_loader)
        #     tgt_f, tgt_y = tgt_iter.next()
            
        total_transfer_loss += jmmd_loss((src_f, F.softmax(src_y, dim=1)), (tgt_f, F.softmax(tgt_y, dim=1)))
    
    return total_transfer_loss / len_dataloader
    

def adapt(encoder, classifier, jmmd_loss, device, src_loader, tgt_loader, optimizer, e, epochs, lambda_coeff):
    encoder.train()
    classifier.train()
    jmmd_loss.train()
    len_dataloader = min(len(src_loader), len(tgt_loader))
    # len_dataloader = max(len(src_loader), len(tgt_loader))
    src_iter = iter(src_loader)
    tgt_iter = iter(tgt_loader)
    
    total_loss = 0
    total_cls_loss = 0
    total_transfer_loss = 0
    total_src_data_size = 0
    
    for i in tqdm(range(len_dataloader)):
        # try:
        src_data = src_iter.next().to(device)
        # except:
        #     src_iter = iter(src_loader)
        #     src_data = src_iter.next().to(device)
            
        # try:
        tgt_data = tgt_iter.next().to(device)
        # except:
        #     tgt_iter = iter(tgt_loader)
        #     tgt_data = tgt_iter.next().to(device)
        
        src_y, src_f = classifier(encoder(src_data.x, src_data.edge_index))
        tgt_y, tgt_f = classifier(encoder(tgt_data.x, tgt_data.edge_index))
        cls_loss = F.nll_loss(F.log_softmax(src_y[src_data.mask], dim=1), src_data.y[src_data.mask])
        # transfer_loss = jmmd_loss((src_f, F.softmax(src_y, dim=1)), (tgt_f, F.softmax(tgt_y, dim=1)))
        transfer_loss = minibatch_jmmd(jmmd_loss, src_f, src_y, tgt_f, tgt_y)
        loss = cls_loss + transfer_loss * lambda_coeff
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * src_data.x.size(0)
        total_cls_loss += cls_loss.item() * src_data.x.size(0)
        total_transfer_loss += transfer_loss.item() * src_data.x.size(0)
        total_src_data_size += src_data.x.size(0)
    
    total_loss /= total_src_data_size
    total_cls_loss /= total_src_data_size
    total_transfer_loss /= total_src_data_size
    return total_loss, total_cls_loss, total_transfer_loss
    
@torch.no_grad()
def adapt_test(encoder, classifier, jmmd_loss, device, src_loader, tgt_loader, e, epochs, lambda_coeff):
    encoder.eval()
    classifier.eval()
    jmmd_loss.eval()
    len_dataloader = min(len(src_loader), len(tgt_loader))
    src_iter = iter(src_loader)
    tgt_iter = iter(tgt_loader)
    
    total_loss = 0
    total_cls_loss = 0
    total_transfer_loss = 0
    total_src_data_size = 0
    
    for i in tqdm(range(len_dataloader)):
        src_data = src_iter.next().to(device)
        
        tgt_data = tgt_iter.next().to(device)
        
        src_y, src_f = classifier(encoder(src_data.x, src_data.edge_index))
        tgt_y, tgt_f = classifier(encoder(tgt_data.x, tgt_data.edge_index))
        cls_loss = F.nll_loss(F.log_softmax(src_y[src_data.mask], dim=1), src_data.y[src_data.mask])
        # transfer_loss = jmmd_loss((src_f, F.softmax(src_y, dim=1)), (tgt_f, F.softmax(tgt_y, dim=1)))
        transfer_loss = minibatch_jmmd(jmmd_loss, src_f, src_y, tgt_f, tgt_y)
        loss = cls_loss + transfer_loss * lambda_coeff
        
        
        total_loss += loss.item() * src_data.x.size(0)
        total_cls_loss += cls_loss.item() * src_data.x.size(0)
        total_transfer_loss += transfer_loss.item() * src_data.x.size(0)
        total_src_data_size += src_data.x.size(0)
    
    total_loss /= total_src_data_size
    total_cls_loss /= total_src_data_size
    total_transfer_loss /= total_src_data_size
    return total_loss, total_cls_loss, total_transfer_loss
    


## Initial Source Stage ##

In [13]:
elliptic_0 = get_data(data_dir, 'elliptic', 0)
feat_dim = elliptic_0.x.shape[1]
hidden_dim = 128
emb_dim = 128
encoder = TwoLayerGraphSAGE(feat_dim, hidden_dim, emb_dim)
mlp = MLPHead(emb_dim, emb_dim // 4, 2)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(encoder.parameters()) + list(mlp.parameters()), lr=1e-3)
epochs = 500
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = encoder.to(device)
mlp = mlp.to(device)

/home/hhchung/dyngraph-uda/elliptic/dataset.py:75: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  edge_index = torch.tensor(A.nonzero(), dtype=torch.long)


In [14]:
split = [0,7,9]
train_data = [get_data(data_dir, 'elliptic', i) for i in range(split[0],split[1])]
val_data = [get_data(data_dir, 'elliptic', i) for i in range(split[1],split[2])]
train_loader = DataLoader(dataset=train_data, batch_size=1, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=1, shuffle=False)

In [15]:
best_f1 = 0
best_encoder = None
best_mlp = None
for e in range(1, epochs + 1):
    train_loss = train(encoder, mlp, optimizer, train_loader, loss_fn, device)
    val_loss, val_f1 = test(encoder, mlp, val_loader, loss_fn, device)
    print(f"Epoch:{e}/{epochs} Train Loss:{round(train_loss,4)} Val Loss:{round(val_loss,4)} Val F1:{round(val_f1, 4)}")
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_encoder = deepcopy(encoder)
        best_mlp = deepcopy(mlp)

encoder = deepcopy(best_encoder)
mlp = deepcopy(best_mlp)

Epoch:1/500 Train Loss:0.3965 Val Loss:0.4848 Val F1:0.0039
Epoch:2/500 Train Loss:0.1371 Val Loss:0.6459 Val F1:0.004
Epoch:3/500 Train Loss:0.0979 Val Loss:0.8028 Val F1:0.0079
Epoch:4/500 Train Loss:0.1012 Val Loss:0.5666 Val F1:0.1328
Epoch:5/500 Train Loss:0.0833 Val Loss:0.3005 Val F1:0.5841
Epoch:6/500 Train Loss:0.0839 Val Loss:0.2747 Val F1:0.5738
Epoch:7/500 Train Loss:0.0655 Val Loss:0.4344 Val F1:0.1344
Epoch:8/500 Train Loss:0.0721 Val Loss:0.5049 Val F1:0.0841
Epoch:9/500 Train Loss:0.0682 Val Loss:0.3919 Val F1:0.1701
Epoch:10/500 Train Loss:0.0585 Val Loss:0.2602 Val F1:0.5697
Epoch:11/500 Train Loss:0.06 Val Loss:0.2372 Val F1:0.6526
Epoch:12/500 Train Loss:0.0514 Val Loss:0.2533 Val F1:0.589
Epoch:13/500 Train Loss:0.0478 Val Loss:0.2584 Val F1:0.5856
Epoch:14/500 Train Loss:0.0441 Val Loss:0.2189 Val F1:0.6512
Epoch:15/500 Train Loss:0.0428 Val Loss:0.2107 Val F1:0.6396
Epoch:16/500 Train Loss:0.0386 Val Loss:0.2036 Val F1:0.6899
Epoch:17/500 Train Loss:0.0388 Val Lo

Epoch:140/500 Train Loss:0.0038 Val Loss:0.2512 Val F1:0.7715
Epoch:141/500 Train Loss:0.003 Val Loss:0.271 Val F1:0.7449
Epoch:142/500 Train Loss:0.0035 Val Loss:0.287 Val F1:0.722
Epoch:143/500 Train Loss:0.0029 Val Loss:0.2908 Val F1:0.7256
Epoch:144/500 Train Loss:0.003 Val Loss:0.3036 Val F1:0.7267
Epoch:145/500 Train Loss:0.0038 Val Loss:0.2901 Val F1:0.7193
Epoch:146/500 Train Loss:0.0028 Val Loss:0.2764 Val F1:0.7448
Epoch:147/500 Train Loss:0.0034 Val Loss:0.2884 Val F1:0.7801
Epoch:148/500 Train Loss:0.0033 Val Loss:0.3036 Val F1:0.7505
Epoch:149/500 Train Loss:0.0025 Val Loss:0.2988 Val F1:0.7433
Epoch:150/500 Train Loss:0.003 Val Loss:0.286 Val F1:0.7362
Epoch:151/500 Train Loss:0.0022 Val Loss:0.3057 Val F1:0.7445
Epoch:152/500 Train Loss:0.0026 Val Loss:0.2751 Val F1:0.7484
Epoch:153/500 Train Loss:0.0021 Val Loss:0.3205 Val F1:0.7296
Epoch:154/500 Train Loss:0.003 Val Loss:0.304 Val F1:0.7562
Epoch:155/500 Train Loss:0.0036 Val Loss:0.2882 Val F1:0.7811
Epoch:156/500 Tra

Epoch:276/500 Train Loss:0.0008 Val Loss:0.4361 Val F1:0.7119
Epoch:277/500 Train Loss:0.0004 Val Loss:0.425 Val F1:0.7204
Epoch:278/500 Train Loss:0.0009 Val Loss:0.4469 Val F1:0.7164
Epoch:279/500 Train Loss:0.0008 Val Loss:0.463 Val F1:0.7325
Epoch:280/500 Train Loss:0.0005 Val Loss:0.4612 Val F1:0.7244
Epoch:281/500 Train Loss:0.0006 Val Loss:0.4514 Val F1:0.7238
Epoch:282/500 Train Loss:0.0007 Val Loss:0.4205 Val F1:0.7492
Epoch:283/500 Train Loss:0.0008 Val Loss:0.458 Val F1:0.7232
Epoch:284/500 Train Loss:0.0003 Val Loss:0.4522 Val F1:0.7435
Epoch:285/500 Train Loss:0.0005 Val Loss:0.4881 Val F1:0.6883
Epoch:286/500 Train Loss:0.0008 Val Loss:0.4762 Val F1:0.7307
Epoch:287/500 Train Loss:0.0003 Val Loss:0.4193 Val F1:0.7496
Epoch:288/500 Train Loss:0.0011 Val Loss:0.4342 Val F1:0.7467
Epoch:289/500 Train Loss:0.0003 Val Loss:0.4411 Val F1:0.7226
Epoch:290/500 Train Loss:0.0002 Val Loss:0.446 Val F1:0.7485
Epoch:291/500 Train Loss:0.0005 Val Loss:0.4711 Val F1:0.7553
Epoch:292/50

Epoch:411/500 Train Loss:0.0004 Val Loss:0.47 Val F1:0.7617
Epoch:412/500 Train Loss:0.0001 Val Loss:0.4435 Val F1:0.7423
Epoch:413/500 Train Loss:0.0001 Val Loss:0.4698 Val F1:0.756
Epoch:414/500 Train Loss:0.0003 Val Loss:0.4614 Val F1:0.7347
Epoch:415/500 Train Loss:0.0003 Val Loss:0.4746 Val F1:0.7489
Epoch:416/500 Train Loss:0.0002 Val Loss:0.4775 Val F1:0.7286
Epoch:417/500 Train Loss:0.0002 Val Loss:0.4673 Val F1:0.7644
Epoch:418/500 Train Loss:0.0003 Val Loss:0.4611 Val F1:0.7839
Epoch:419/500 Train Loss:0.0001 Val Loss:0.4521 Val F1:0.7799
Epoch:420/500 Train Loss:0.0001 Val Loss:0.4733 Val F1:0.7563
Epoch:421/500 Train Loss:0.0001 Val Loss:0.4512 Val F1:0.7494
Epoch:422/500 Train Loss:0.0001 Val Loss:0.484 Val F1:0.765
Epoch:423/500 Train Loss:0.0001 Val Loss:0.4386 Val F1:0.7663
Epoch:424/500 Train Loss:0.0001 Val Loss:0.4611 Val F1:0.7514
Epoch:425/500 Train Loss:0.0001 Val Loss:0.4744 Val F1:0.766
Epoch:426/500 Train Loss:0.0003 Val Loss:0.4754 Val F1:0.761
Epoch:427/500 T

In [16]:
print(best_f1)

0.796399217221135


## Prequential Evaluation on Subsequent Time Steps ##

In [17]:
def continual_adapt(split, src_train_loader, src_val_loader, encoder, mlp, device, lambda_coeff=1, lr=1e-3):
    tgt_train_loader = DataLoader(dataset=[get_data(data_dir, "elliptic", i) for i in range(split[0],split[1])], batch_size=1, shuffle=True)
    tgt_val_loader = DataLoader(dataset=[get_data(data_dir, "elliptic", i) for i in range(split[1],split[2])], batch_size=1, shuffle=False)
    thetas = None # none adversarial
    jmmd_loss = JointMultipleKernelMaximumMeanDiscrepancy(
        kernels=(
            [GaussianKernel(alpha=2 ** k) for k in range(-3, 2)],
            (GaussianKernel(sigma=0.92, track_running_stats=False),)
        ),
        linear=False, thetas=thetas
    ).to(device)
    tgt_encoder, tgt_mlp = deepcopy(encoder), deepcopy(mlp)
    tgt_optimizer = torch.optim.Adam(list(tgt_encoder.parameters()) + list(tgt_mlp.parameters()), lr=lr)
    
    epochs = 1000
    best_val_loss = np.inf
    best_val_cls_loss, best_val_transfer_loss = None, None
    best_tgt_encoder, best_tgt_mlp = None, None
    patience = 10
    staleness = 0

    for e in range(1, epochs + 1):
        total_train_loss, total_train_cls_loss, total_train_transfer_loss = adapt(tgt_encoder, tgt_mlp, jmmd_loss, device, src_train_loader, tgt_train_loader, tgt_optimizer, e, epochs, lambda_coeff)
        total_val_loss, total_val_cls_loss, total_val_transfer_loss = adapt_test(tgt_encoder, tgt_mlp, jmmd_loss, device, src_val_loader, tgt_val_loader, e, epochs, lambda_coeff)
        if total_val_loss < best_val_loss:
            best_val_loss = total_val_loss
            best_val_cls_loss = total_val_cls_loss
            best_val_transfer_loss = total_val_transfer_loss
            best_tgt_encoder = deepcopy(tgt_encoder)
            best_tgt_mlp = deepcopy(tgt_mlp)
            staleness = 0
        else:
            staleness += 1
        print(f'Epoch {e}/{epochs} Train Total Loss: {round(total_train_loss,3)} Train Src Cls Loss: {round(total_train_cls_loss,3)} Train Tgt Transfer Loss: {round(total_train_transfer_loss,3)} \n Val Total Loss: {round(total_val_loss,3)} Val Src Cls Loss: {round(total_val_cls_loss,3)} Val Tgt Transfer Loss: {round(total_val_transfer_loss,3)}')

        if staleness > patience:
            break

    tgt_encoder = deepcopy(best_tgt_encoder)
    tgt_mlp = deepcopy(best_tgt_mlp)
    
    return tgt_encoder, tgt_mlp, best_val_loss, best_val_cls_loss, best_val_transfer_loss

In [18]:
lambda_coeff = 0.1
lr = 1e-4

In [19]:
f1_list = []
f1_list.append(best_f1)

### 9-13 ###

In [20]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(9,14)]
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [21]:
test_loss, test_f1 = test(encoder, mlp, test_loader, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 0.2805, Test F1: 0.6911


In [22]:
encoder_9_14, mlp_9_14, best_val_loss, best_val_cls_loss, best_val_transfer_loss = continual_adapt([7,12,14], train_loader, val_loader, encoder, mlp, device, lambda_coeff=lambda_coeff, lr=lr)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 29.09it/s]


Epoch 1/1000 Train Total Loss: 0.024 Train Src Cls Loss: 0.006 Train Tgt Transfer Loss: 0.176 
 Val Total Loss: 0.24 Val Src Cls Loss: 0.227 Val Tgt Transfer Loss: 0.133


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 29.13it/s]


Epoch 2/1000 Train Total Loss: 0.025 Train Src Cls Loss: 0.009 Train Tgt Transfer Loss: 0.163 
 Val Total Loss: 0.232 Val Src Cls Loss: 0.217 Val Tgt Transfer Loss: 0.147


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 28.94it/s]


...

Epoch 18/1000 Train Total Loss: 0.016 Train Src Cls Loss: 0.01 Train Tgt Transfer Loss: 0.068 
 Val Total Loss: 0.248 Val Src Cls Loss: 0.241 Val Tgt Transfer Loss: 0.074


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 28.28it/s]


Epoch 19/1000 Train Total Loss: 0.016 Train Src Cls Loss: 0.009 Train Tgt Transfer Loss: 0.067 
 Val Total Loss: 0.256 Val Src Cls Loss: 0.249 Val Tgt Transfer Loss: 0.07


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 27.85it/s]

Epoch 20/1000 Train Total Loss: 0.015 Train Src Cls Loss: 0.007 Train Tgt Transfer Loss: 0.074 
 Val Total Loss: 0.249 Val Src Cls Loss: 0.243 Val Tgt Transfer Loss: 0.065


In [23]:
print(f'Total Val Loss: {round(best_val_loss, 3)}, Val Cls Loss: {round(best_val_cls_loss, 3)}, Val Transfer Loss: {round(best_val_transfer_loss, 3)}')

Total Val Loss: 0.218, Val Cls Loss: 0.209, Val Transfer Loss: 0.095


### 14-18 ###

In [24]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(14,19)]
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [25]:
test_loss, test_f1 = test(encoder_9_14, mlp_9_14, test_loader, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 0.3726, Test F1: 0.736


In [26]:
encoder_14_19, mlp_14_19, best_val_loss, best_val_cls_loss, best_val_transfer_loss = continual_adapt([12,17,19], train_loader, val_loader, encoder_9_14, mlp_9_14, device, lambda_coeff=lambda_coeff, lr=lr)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 30.91it/s]


Epoch 1/1000 Train Total Loss: 0.037 Train Src Cls Loss: 0.005 Train Tgt Transfer Loss: 0.321 
 Val Total Loss: 0.228 Val Src Cls Loss: 0.217 Val Tgt Transfer Loss: 0.116


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 32.25it/s]


Epoch 2/1000 Train Total Loss: 0.03 Train Src Cls Loss: 0.006 Train Tgt Transfer Loss: 0.236 
 Val Total Loss: 0.253 Val Src Cls Loss: 0.242 Val Tgt Transfer Loss: 0.106


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 15.84it/s]


...

Epoch 10/1000 Train Total Loss: 0.019 Train Src Cls Loss: 0.008 Train Tgt Transfer Loss: 0.114 
 Val Total Loss: 0.273 Val Src Cls Loss: 0.267 Val Tgt Transfer Loss: 0.052


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 15.24it/s]


Epoch 11/1000 Train Total Loss: 0.016 Train Src Cls Loss: 0.006 Train Tgt Transfer Loss: 0.102 
 Val Total Loss: 0.274 Val Src Cls Loss: 0.268 Val Tgt Transfer Loss: 0.058


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 15.42it/s]

Epoch 12/1000 Train Total Loss: 0.02 Train Src Cls Loss: 0.011 Train Tgt Transfer Loss: 0.093 
 Val Total Loss: 0.283 Val Src Cls Loss: 0.279 Val Tgt Transfer Loss: 0.035


In [27]:
print(f'Total Val Loss: {round(best_val_loss, 3)}, Val Cls Loss: {round(best_val_cls_loss, 3)}, Val Transfer Loss: {round(best_val_transfer_loss, 3)}')

Total Val Loss: 0.228, Val Cls Loss: 0.217, Val Transfer Loss: 0.116


### 19-23 ###

In [28]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(19,24)]
test_loader_19_24 = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [29]:
test_loss, test_f1 = test(encoder_14_19, mlp_14_19, test_loader_19_24, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 0.9078, Test F1: 0.4623


In [30]:
encoder_19_24, mlp_19_24, best_val_loss, best_val_cls_loss, best_val_transfer_loss = continual_adapt([17,22,24], train_loader, val_loader, encoder_14_19, mlp_14_19, device, lambda_coeff=lambda_coeff, lr=lr)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 13.01it/s]


Epoch 1/1000 Train Total Loss: 0.065 Train Src Cls Loss: 0.005 Train Tgt Transfer Loss: 0.599 
 Val Total Loss: 0.285 Val Src Cls Loss: 0.246 Val Tgt Transfer Loss: 0.387


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 13.20it/s]


Epoch 2/1000 Train Total Loss: 0.058 Train Src Cls Loss: 0.008 Train Tgt Transfer Loss: 0.501 
 Val Total Loss: 0.268 Val Src Cls Loss: 0.237 Val Tgt Transfer Loss: 0.316


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 13.29it/s]


...

Epoch 36/1000 Train Total Loss: 0.013 Train Src Cls Loss: 0.007 Train Tgt Transfer Loss: 0.059 
 Val Total Loss: 0.259 Val Src Cls Loss: 0.256 Val Tgt Transfer Loss: 0.034


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 13.49it/s]


Epoch 37/1000 Train Total Loss: 0.018 Train Src Cls Loss: 0.013 Train Tgt Transfer Loss: 0.055 
 Val Total Loss: 0.262 Val Src Cls Loss: 0.258 Val Tgt Transfer Loss: 0.032


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 23.39it/s]

Epoch 38/1000 Train Total Loss: 0.012 Train Src Cls Loss: 0.008 Train Tgt Transfer Loss: 0.041 
 Val Total Loss: 0.274 Val Src Cls Loss: 0.27 Val Tgt Transfer Loss: 0.034


In [31]:
print(f'Total Val Loss: {round(best_val_loss, 3)}, Val Cls Loss: {round(best_val_cls_loss, 3)}, Val Transfer Loss: {round(best_val_transfer_loss, 3)}')

Total Val Loss: 0.249, Val Cls Loss: 0.244, Val Transfer Loss: 0.043


### 24-28 ###

In [32]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(24,29)]
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [33]:
test_loss, test_f1 = test(encoder_19_24, mlp_19_24, test_loader, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 1.1421, Test F1: 0.5649


In [34]:
encoder_24_29, mlp_24_29, best_val_loss, best_val_cls_loss, best_val_transfer_loss = continual_adapt([22,27,29], train_loader, val_loader, encoder_19_24, mlp_19_24, device, lambda_coeff=lambda_coeff, lr=lr)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 14.90it/s]


Epoch 1/1000 Train Total Loss: 0.022 Train Src Cls Loss: 0.007 Train Tgt Transfer Loss: 0.152 
 Val Total Loss: 0.291 Val Src Cls Loss: 0.272 Val Tgt Transfer Loss: 0.183


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 14.82it/s]


Epoch 2/1000 Train Total Loss: 0.025 Train Src Cls Loss: 0.011 Train Tgt Transfer Loss: 0.133 
 Val Total Loss: 0.281 Val Src Cls Loss: 0.267 Val Tgt Transfer Loss: 0.141


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 14.86it/s]


...

Epoch 23/1000 Train Total Loss: 0.017 Train Src Cls Loss: 0.012 Train Tgt Transfer Loss: 0.053 
 Val Total Loss: 0.28 Val Src Cls Loss: 0.275 Val Tgt Transfer Loss: 0.052


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 14.84it/s]


Epoch 24/1000 Train Total Loss: 0.014 Train Src Cls Loss: 0.011 Train Tgt Transfer Loss: 0.034 
 Val Total Loss: 0.271 Val Src Cls Loss: 0.266 Val Tgt Transfer Loss: 0.052


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 29.77it/s]

Epoch 25/1000 Train Total Loss: 0.014 Train Src Cls Loss: 0.01 Train Tgt Transfer Loss: 0.036 
 Val Total Loss: 0.271 Val Src Cls Loss: 0.265 Val Tgt Transfer Loss: 0.056


In [35]:
print(f'Total Val Loss: {round(best_val_loss, 3)}, Val Cls Loss: {round(best_val_cls_loss, 3)}, Val Transfer Loss: {round(best_val_transfer_loss, 3)}')

Total Val Loss: 0.249, Val Cls Loss: 0.241, Val Transfer Loss: 0.073


### 29-33 ###

In [36]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(29,34)]
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [37]:
test_loss, test_f1 = test(encoder_24_29, mlp_24_29, test_loader, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 1.6545, Test F1: 0.3401


In [38]:
encoder_29_34, mlp_29_34, best_val_loss, best_val_cls_loss, best_val_transfer_loss = continual_adapt([27,32,34], train_loader, val_loader, encoder_24_29, mlp_24_29, device, lambda_coeff=lambda_coeff, lr=lr)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 29.60it/s]


Epoch 1/1000 Train Total Loss: 0.027 Train Src Cls Loss: 0.013 Train Tgt Transfer Loss: 0.134 
 Val Total Loss: 0.267 Val Src Cls Loss: 0.252 Val Tgt Transfer Loss: 0.153


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 30.77it/s]


Epoch 2/1000 Train Total Loss: 0.021 Train Src Cls Loss: 0.011 Train Tgt Transfer Loss: 0.095 
 Val Total Loss: 0.284 Val Src Cls Loss: 0.272 Val Tgt Transfer Loss: 0.121


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 14.93it/s]


...

Epoch 10/1000 Train Total Loss: 0.014 Train Src Cls Loss: 0.008 Train Tgt Transfer Loss: 0.056 
 Val Total Loss: 0.298 Val Src Cls Loss: 0.292 Val Tgt Transfer Loss: 0.062


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 15.31it/s]


Epoch 11/1000 Train Total Loss: 0.014 Train Src Cls Loss: 0.008 Train Tgt Transfer Loss: 0.058 
 Val Total Loss: 0.281 Val Src Cls Loss: 0.276 Val Tgt Transfer Loss: 0.055


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 31.36it/s]

Epoch 12/1000 Train Total Loss: 0.02 Train Src Cls Loss: 0.014 Train Tgt Transfer Loss: 0.064 
 Val Total Loss: 0.301 Val Src Cls Loss: 0.296 Val Tgt Transfer Loss: 0.047


In [39]:
print(f'Total Val Loss: {round(best_val_loss, 3)}, Val Cls Loss: {round(best_val_cls_loss, 3)}, Val Transfer Loss: {round(best_val_transfer_loss, 3)}')

Total Val Loss: 0.267, Val Cls Loss: 0.252, Val Transfer Loss: 0.153


### 34-38 ###

In [40]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(34,39)]
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [41]:
test_loss, test_f1 = test(encoder_29_34, mlp_29_34, test_loader, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 1.7673, Test F1: 0.3215


In [42]:
encoder_34_39, mlp_34_39, best_val_loss, best_val_cls_loss, best_val_transfer_loss = continual_adapt([32,37,39], train_loader, val_loader, encoder_29_34, mlp_29_34, device, lambda_coeff=lambda_coeff, lr=lr)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 14.89it/s]


Epoch 1/1000 Train Total Loss: 0.038 Train Src Cls Loss: 0.014 Train Tgt Transfer Loss: 0.239 
 Val Total Loss: 0.284 Val Src Cls Loss: 0.251 Val Tgt Transfer Loss: 0.33


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 31.31it/s]


Epoch 2/1000 Train Total Loss: 0.04 Train Src Cls Loss: 0.015 Train Tgt Transfer Loss: 0.243 
 Val Total Loss: 0.279 Val Src Cls Loss: 0.253 Val Tgt Transfer Loss: 0.262


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 30.66it/s]


...

Epoch 11/1000 Train Total Loss: 0.021 Train Src Cls Loss: 0.015 Train Tgt Transfer Loss: 0.06 
 Val Total Loss: 0.302 Val Src Cls Loss: 0.287 Val Tgt Transfer Loss: 0.15


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 31.79it/s]


Epoch 12/1000 Train Total Loss: 0.019 Train Src Cls Loss: 0.014 Train Tgt Transfer Loss: 0.046 
 Val Total Loss: 0.301 Val Src Cls Loss: 0.288 Val Tgt Transfer Loss: 0.127


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 32.26it/s]

Epoch 13/1000 Train Total Loss: 0.018 Train Src Cls Loss: 0.013 Train Tgt Transfer Loss: 0.052 
 Val Total Loss: 0.308 Val Src Cls Loss: 0.295 Val Tgt Transfer Loss: 0.125


In [43]:
print(f'Total Val Loss: {round(best_val_loss, 3)}, Val Cls Loss: {round(best_val_cls_loss, 3)}, Val Transfer Loss: {round(best_val_transfer_loss, 3)}')

Total Val Loss: 0.279, Val Cls Loss: 0.253, Val Transfer Loss: 0.262


### 39-43 ###

In [44]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(39,44)]
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [45]:
test_loss, test_f1 = test(encoder_34_39, mlp_34_39, test_loader, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 3.1642, Test F1: 0.1401


In [46]:
encoder_39_44, mlp_39_44, best_val_loss, best_val_cls_loss, best_val_transfer_loss = continual_adapt([37,42,44], train_loader, val_loader, encoder_34_39, mlp_34_39, device, lambda_coeff=lambda_coeff, lr=lr)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 21.48it/s]


Epoch 1/1000 Train Total Loss: 0.051 Train Src Cls Loss: 0.014 Train Tgt Transfer Loss: 0.373 
 Val Total Loss: 0.269 Val Src Cls Loss: 0.257 Val Tgt Transfer Loss: 0.12


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 12.38it/s]


Epoch 2/1000 Train Total Loss: 0.046 Train Src Cls Loss: 0.018 Train Tgt Transfer Loss: 0.272 
 Val Total Loss: 0.286 Val Src Cls Loss: 0.276 Val Tgt Transfer Loss: 0.098


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 12.41it/s]


...

Epoch 17/1000 Train Total Loss: 0.018 Train Src Cls Loss: 0.013 Train Tgt Transfer Loss: 0.057 
 Val Total Loss: 0.294 Val Src Cls Loss: 0.29 Val Tgt Transfer Loss: 0.043


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 12.33it/s]


Epoch 18/1000 Train Total Loss: 0.022 Train Src Cls Loss: 0.016 Train Tgt Transfer Loss: 0.061 
 Val Total Loss: 0.309 Val Src Cls Loss: 0.304 Val Tgt Transfer Loss: 0.047


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 12.08it/s]

Epoch 19/1000 Train Total Loss: 0.021 Train Src Cls Loss: 0.017 Train Tgt Transfer Loss: 0.041 
 Val Total Loss: 0.308 Val Src Cls Loss: 0.303 Val Tgt Transfer Loss: 0.042


In [47]:
print(f'Total Val Loss: {round(best_val_loss, 3)}, Val Cls Loss: {round(best_val_cls_loss, 3)}, Val Transfer Loss: {round(best_val_transfer_loss, 3)}')

Total Val Loss: 0.269, Val Cls Loss: 0.263, Val Transfer Loss: 0.056


### 44-48 ###

In [48]:
test_data = [get_data(data_dir, "elliptic", i) for i in range(44,49)]
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [49]:
test_loss, test_f1 = test(encoder_39_44, mlp_39_44, test_loader, loss_fn, device)
f1_list.append(test_f1)
print(f"Test Loss: {round(test_loss,4)}, Test F1: {round(test_f1,4)}")

Test Loss: 1.2132, Test F1: 0.1149


In [50]:
sum(f1_list) / len(f1_list)

0.4630249896219412